In [1]:
import numpy as np
import pandas as pd
import pymysql
import pymysql.cursors
from collections import Counter
import pickle as pkl
from config import config_dict as c
import warnings
warnings.filterwarnings('ignore')

In [2]:
def call_db(sql):

    connecting = pymysql.connect(host=c['host'], port=c['port'],
                                 user=c['user'], passwd=c['passwd'],
                                 db=c['db'], charset='utf8')
    curs = connecting.cursor(pymysql.cursors.DictCursor)
    curs.execute(sql)
    rows = curs.fetchall()
    df = pd.DataFrame(data=rows)

    return df

In [3]:
gamecontapp = call_db('select GMKEY, HOW, FIELD, HITNAME, HITTER,\
                      PITNAME, PITCHER from gamecontapp where GYEAR < 2019')

In [4]:
gamecontapp_copy = gamecontapp.copy()

In [5]:
counts = {'batter': {}, 'pitcher':{}}
counts['batter'] = dict(gamecontapp_copy.HITTER.value_counts())
counts['pitcher'] = dict(gamecontapp_copy.PITCHER.value_counts())

# delete hitter & pitcher by cutoff index

In [6]:
cutoffs = {}
percentile_cutoff = 0.9
for player_type in ['batter', 'pitcher']:
    counts_list = list(counts[player_type].values())
    counts_list.sort(reverse = True)
    total_at_bats = sum(counts_list)
    
    cumulative_percentage = [sum(counts_list[:i + 1]) / total_at_bats for i in range(len(counts_list))]
    cutoff_index = sum([1 for total in cumulative_percentage if total <= percentile_cutoff])
    
    cutoff = counts_list[cutoff_index]
    cutoffs[player_type] = cutoff
    print(len(counts[player_type]), cutoff_index, cutoff_index / len(counts[player_type]))

1651 488 0.29557843731072075
1282 528 0.4118564742589704


In [7]:
cutoffs

{'batter': 938, 'pitcher': 891}

In [8]:
batter_name = list(gamecontapp_copy.HITNAME.value_counts()[gamecontapp_copy.HITNAME.value_counts() > cutoffs['batter']].index)
pitcher_name = list(gamecontapp_copy.PITNAME.value_counts()[gamecontapp_copy.PITNAME.value_counts() > cutoffs['pitcher']].index)
batter_pcode = list(gamecontapp_copy.HITTER.value_counts()[gamecontapp_copy.HITTER.value_counts() > cutoffs['batter']].index)
pitcher_pcode = list(gamecontapp_copy.PITCHER.value_counts()[gamecontapp_copy.PITCHER.value_counts() > cutoffs['pitcher']].index)

In [9]:
len(batter_name), len(batter_pcode), len(pitcher_name), len(pitcher_pcode)

(480, 488, 538, 528)

In [10]:
bat_idx = []
for i in range(len(gamecontapp_copy)):
    if gamecontapp_copy.HITTER[i] in batter_pcode:
        bat_idx.append(i)
        
gamecontapp_copy = gamecontapp_copy.loc[bat_idx].reset_index(drop=True)

In [11]:
pit_idx = []
for j in range(len(gamecontapp_copy)):
    if gamecontapp_copy.PITCHER[j] in pitcher_pcode:
        pit_idx.append(j)
        
gamecontapp_copy = gamecontapp_copy.loc[pit_idx].reset_index(drop=True)

In [12]:
len(pit_idx), len(bat_idx)

(1610499, 1786919)

In [13]:
gamecontapp_copy.shape # cutoff 0.9 / final raw data

(1610499, 7)

# features

In [14]:
train = pd.DataFrame(columns=['batter', 'outcome', 'pitcher'])

In [15]:
gamecontapp_copy.HOW.replace({'BN':'OUT', 'GR':'OUT', 'FL':'OUT'}, inplace=True)

In [16]:
HOW_list = ['BB', 'H1', 'H2', 'H3', 'HR', 'HP', 'IB', 'KK', 'BK', 'OUT']
OUT_HOW_list = ['BN', 'GR', 'FL']
HOW_dir_list = ['H1', 'H2', 'H3', 'HR']

# drop not how_list

In [17]:
no_how_ls = []
for i in range(len(gamecontapp_copy)):
    if gamecontapp_copy.HOW[i] not in HOW_list:
        no_how_ls.append(i)

In [18]:
gamecontapp_copy = gamecontapp_copy.drop(index=no_how_ls).reset_index(drop=True) # drop not HOW_list

In [19]:
gamecontapp_copy.shape

(1025441, 7)

# make outcomes

 - N    주자 생존 (보살x 자살x)
 - A    1루 베이스 아웃
 - B    2루 베이스 아웃
 - C    3루 베이스 아웃
 - D    홈 베이스 아웃
 - T    태그 아웃
 - G    땅볼
 - F    플라이
 - P    파울플라이
 - E    실책
 - L    라인드라이브
 - M    디플렉트(타구가 야수의 몸 또는 장비에 맞아 타구 방향이 본래의 진로에서 벗어나거나 속도가 변하는 것)

In [20]:
import re

In [21]:
outcome = []
temp = []

In [22]:
gamecontapp_copy.replace('??', '', inplace=True)
gamecontapp_copy.replace('IN', '', inplace=True)

In [23]:
for i in range(len(gamecontapp_copy)):
    if gamecontapp_copy.FIELD[i] == '' or gamecontapp_copy.FIELD[i] == ' ' or gamecontapp_copy.FIELD[i] == None:
        outcome.append(gamecontapp_copy.HOW[i])
    elif gamecontapp_copy.HOW[i] == 'OUT' or gamecontapp_copy.HOW[i] == 'H1' or gamecontapp_copy.HOW[i] == 'H2' or gamecontapp_copy.HOW[i] == 'H3' or gamecontapp_copy.HOW[i] == 'HR':
        regex = re.compile(r'\d')
        matchobj = regex.search(gamecontapp_copy.FIELD[i])
        temp = gamecontapp_copy.HOW[i] + '_' + matchobj.group()
        outcome.append(temp)
    else:
        outcome.append(gamecontapp_copy.HOW[i])

In [24]:
gamecontapp_copy['outcome'] = outcome

# insert in train df

In [25]:
len(gamecontapp_copy), len(gamecontapp_copy.HITTER.unique()), len(gamecontapp_copy.PITCHER.unique()), len(gamecontapp_copy.outcome.unique())

(1025441, 488, 528, 43)

In [26]:
train['batter'] = gamecontapp_copy.HITTER.values

hitidx = []
hitcode = []
hit_dict = {}
for idx, val in enumerate(train.batter.unique()):
    hitidx.append(idx)
    hitcode.append(val)

for i in range(len(hitcode)):
    hit_dict[i] = hitcode[i]

new_hit_dict = {}
for k,v in hit_dict.items():
#     print(k, v)
    new_hit_dict[v] = k

train.batter.replace(new_hit_dict, inplace=True)

In [27]:
train['pitcher'] = gamecontapp_copy.PITCHER.values

pitidx = []
pitcode = []
pit_dict = {}
for idx, val in enumerate(train.pitcher.unique()):
    pitidx.append(idx)
    pitcode.append(val)

for i in range(len(pitcode)):
    pit_dict[i] = pitcode[i]

new_pit_dict = {}
for k,v in pit_dict.items():
#     print(k, v)
    new_pit_dict[v] = k

train.pitcher.replace(new_pit_dict, inplace=True)

In [28]:
train['outcome'] = outcome

In [29]:
# at_bat outcome categories
len(train.outcome.unique())

43